In [14]:
import pandas as pd
from collections import Counter
from underthesea import word_tokenize  # Sử dụng underthesea để tokenize tiếng Việt

# Đọc dữ liệu từ file
df = pd.read_csv('Comment_data.csv')

# Loại bỏ giá trị null trong cột 'content_cmt'
df = df.dropna(subset=['content_cmt'])

In [15]:
# Loại bỏ khoảng trắng dư thừa ở đầu và cuối chuỗi
df['content_cmt'] = df['content_cmt'].str.strip()

# Thay thế các chuỗi rỗng hoặc chỉ gồm khoảng trắng bằng NaN
df['content_cmt'].replace('', pd.NA, inplace=True)

# Loại bỏ các giá trị NaN
df.dropna(subset=['content_cmt'], inplace=True)

# Kiểm tra lại DataFrame để đảm bảo rằng không còn giá trị rỗng
df['content_cmt']

C:\Users\HELLO\AppData\Local\Temp\ipykernel_5636\2218818010.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['content_cmt'].replace('', pd.NA, inplace=True)


0        hàng_chính_hãng flagship store hồng_siêu_iu gó...
1        chất_lượng giao_hàng nhanh sản_phẩm đúng chính...
2                 chuẩn giao_hàng nhanh hình_ảnh tính_chất
3                    giao_hàng đúng hẹn chất_lượng máy tốt
4                                            hồng cưng xiu
                               ...                        
23870                                               qa tốt
23871                                               cay cú
23872                                 iphone khuyên iphone
23873                                                  bắt
23874                                 hàng check hàng toán
Name: content_cmt, Length: 23828, dtype: object

In [16]:
from underthesea import ner
# Hàm để gán nhãn cho mỗi câu
def label_sentence(sentence):
    # Thực hiện NER
    entities = ner(sentence)
    return entities

# Khởi tạo cột 'label' với giá trị None cho mỗi hàng
df['label'] = None

# Tối ưu bằng cách sử dụng apply thay vì vòng lặp
df['label'] = df['content_cmt'].apply(label_sentence)

In [17]:
data = df.copy()

In [18]:
entries = [
    (index, entry[0], entry[1], entry[2], entry[3], row['content_cmt'])
    for index, row in data.iterrows()
    for entry in row['label']
]

# Nếu 'entries' không rỗng, chuyển đổi thành DataFrame, ngược lại tạo DataFrame trống với các cột tương ứng
if entries:
    expanded_data = pd.DataFrame(entries, columns=['Sentence ID', 'Word', 'POS Tag', 'Chunk Tag', 'NE Label', 'Content Cmt'])
else:
    expanded_data = pd.DataFrame(columns=['Sentence ID', 'Word', 'POS Tag', 'Chunk Tag', 'NE Label', 'Content Cmt'])


In [19]:
expanded_data

,Sentence ID,Word,POS Tag,Chunk Tag,NE Label,Content Cmt
0,0,hàng_chính_hãng,N,B-NP,O,hàng_chính_hãng flagship store hồng_siêu_iu gó...
1,0,flagship,V,B-VP,O,hàng_chính_hãng flagship store hồng_siêu_iu gó...
2,0,store,N,B-NP,O,hàng_chính_hãng flagship store hồng_siêu_iu gó...
3,0,hồng_siêu_iu,V,B-VP,O,hàng_chính_hãng flagship store hồng_siêu_iu gó...
4,0,gói_hàng,N,B-NP,O,hàng_chính_hãng flagship store hồng_siêu_iu gó...
...,...,...,...,...,...,...
134194,23873,bắt,V,B-VP,O,bắt
134195,23874,hàng,N,B-NP,O,hàng check hàng toán
134196,23874,check,V,B-VP,O,hàng check hàng toán
134197,23874,hàng,N,B-NP,O,hàng check hàng toán


In [20]:
# Lưu kết quả vào một tệp mới
expanded_data.to_csv('label_dataset_ner.csv', index=False, encoding='utf-8-sig')

In [77]:
import pandas as pd
data = pd.read_csv('label_dataset_ner.csv')
label = pd.read_csv('DT.csv')

In [78]:
data.head()

,Sentence ID,Word,POS Tag,Chunk Tag,NE Label,Content Cmt
0,0,hàng_chính_hãng,N,B-NP,O,hàng_chính_hãng flagship store hồng_siêu_iu gó...
1,0,flagship,V,B-VP,O,hàng_chính_hãng flagship store hồng_siêu_iu gó...
2,0,store,N,B-NP,O,hàng_chính_hãng flagship store hồng_siêu_iu gó...
3,0,hồng_siêu_iu,V,B-VP,O,hàng_chính_hãng flagship store hồng_siêu_iu gó...
4,0,gói_hàng,N,B-NP,O,hàng_chính_hãng flagship store hồng_siêu_iu gó...


In [79]:
# Chuyển đổi dataframe label thành danh sách các từ tính từ và danh từ
ADJ = label['ADJ'].dropna().tolist()  # Loại bỏ các giá trị NaN và chuyển thành list
N = label['N'].dropna().tolist()  # Tương tự cho danh từ
ORG = label['ORG'].dropna().tolist()


In [80]:

# Hàm để thay đổi cột POS và NE Label
def update_pos_ne_label(row):
    if row['Word'] in ADJ:
        row['POS Tag'] = 'ADJ'
        row['NE Label'] = 'B-ADJ'
    elif row['Word'] in N:
        row['POS Tag'] = 'N'
        row['NE Label'] = 'B-OBJ'
    elif row['Word'] in ORG:
        row['POS Tag'] = 'N'
        row['NE Label'] = 'B-ORG'
    return row

# Áp dụng hàm lên dataframe
data = data.apply(update_pos_ne_label, axis=1)


In [81]:
data.head()

,Sentence ID,Word,POS Tag,Chunk Tag,NE Label,Content Cmt
0,0,hàng_chính_hãng,N,B-NP,B-OBJ,hàng_chính_hãng flagship store hồng_siêu_iu gó...
1,0,flagship,N,B-VP,B-ORG,hàng_chính_hãng flagship store hồng_siêu_iu gó...
2,0,store,N,B-NP,O,hàng_chính_hãng flagship store hồng_siêu_iu gó...
3,0,hồng_siêu_iu,V,B-VP,O,hàng_chính_hãng flagship store hồng_siêu_iu gó...
4,0,gói_hàng,N,B-NP,B-OBJ,hàng_chính_hãng flagship store hồng_siêu_iu gó...


In [25]:
# Hàm để thay đổi NE Label dựa trên các quy tắc mới cho từ 'hàng', 'chính', 'hãng' và 'gói'
def update_ne_label(data):
    # Duyệt qua mỗi câu
    for _, group in data.groupby('sentences'):
        words = group['word'].tolist()
        for i, (index, row) in enumerate(group.iterrows()):
            if row['word'] == 'hàng':
                if i == 0:  # Từ 'hàng' ở vị trí đầu của câu
                    data.at[index, 'NE Label'] = 'B-OBJ'
                else:  # Từ 'hàng' không phải ở vị trí đầu của câu
                    data.at[index, 'NE Label'] = 'I-OBJ'
            elif row['word'] == 'chính' and (i > 0 and words[i-1] == 'hàng'):
                data.at[index, 'NE Label'] = 'I-OBJ'
            elif row['word'] == 'hãng' and (i > 0 and words[i-1] == 'chính'):
                data.at[index, 'NE Label'] = 'I-OBJ'
            elif row['word'] == 'gói':
                data.at[index, 'NE Label'] = 'B-OBJ'
            elif row['word'] == 'máy':
                data.at[index, 'NE Label'] = 'B-OBJ'  
    return data

# Áp dụng hàm cập nhật NE Label cho DataFrame
data = update_ne_label(data)

In [82]:
data.to_csv('label_ner.csv', index=False, encoding='utf-8-sig')